From the image of the montly histories, produce feature collections and, ultimately, CSV with their reduction over city outlines.

In [9]:
import ee
import geemap
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
ee.Initialize()

In [4]:
# Load the cities feature collection and filter for South American countries
southAmericaISO = ['ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'GUY', 'PRY', 'PER', 'SUR', 'URY', 'VEN'];
cities = ee.FeatureCollection("projects/dw-city-tree-coverage/assets/city_outlines")\
              .filter(ee.Filter.inList('CTR_MN_ISO', southAmericaISO))\
              .filter(ee.Filter.gte('P15', 7e4));


In [6]:
# For each available month history, access the respective data and reduce it to a region.
# Then, cast to pandas and save locally.
# This only needs to run once.
for month in range(1, 13):
    month_history = ee.Image(f"projects/ee-tree-coverage-cities/assets/pm2p5-monthly-2017-2023/Month_{month}_Mean")
    
    city_month_history = month_history.reduceRegions(
        collection=cities,                  # The cities feature collection
        reducer=ee.Reducer.mean(),          # Use mean reducer
        scale=44528,                        # Adjust scale based on the dataset resolution
    ).map(lambda feature: feature.set({
        'month': month_history.get('month'),
        'lat': feature.geometry().centroid().coordinates().get(1),  # Extract latitude
        'lon': feature.geometry().centroid().coordinates().get(0),  # Extract longitude
    }))
    
    city_month_history = geemap.ee_to_df(city_month_history)
        
    city_month_history.to_csv(f"../output/city-monthly-means/City_Month_{month}_Mean", index=False)